In [1]:
import os
import pandas as pd
from darts import TimeSeries
import matplotlib.pyplot as plt
from darts.metrics import mape, mae, rmse




In [2]:
## read data
def get_timeseries (filepath):
    electricity_consumption = pd.read_csv(filepath)
    # Convert dataframe to Timeseries
    ts = TimeSeries.from_dataframe(electricity_consumption, time_col="Date" )
    return ts


In [3]:
def eval_model(pred_series, full_data,val_target, DESC):
    plt.figure(figsize=(8, 5))
    full_data[DESC].plot(label="actual")
    pred_series[DESC].plot(label="forecast")
    plt.title("Seasonal Naive Bayes Model: RMSE: {:.2f}, MAE: {:.2f}, MAPE: {:.2f}".format(rmse(val_target, pred_series), mae(val_target, pred_series), mape(val_target, pred_series)))
    plt.legend()

# Read True Data

In [4]:
file_list_name = ['Electricity-ResBusClimateUNI.csv', 'Electricity-ResBusClimateCNI.csv', 
                  'Electricity-ResBusClimateLNI.csv', 'Electricity-ResBusClimateUSI.csv', 'Electricity-ResBusClimateLSI.csv']
path_name = '../AnalysisAndPreprocessing/ProcessedData_Original/'
timeseries_list = [get_timeseries(path_name + file) for file in file_list_name]
TRUE_ORDER = ['UNI', 'CNI', 'LNI', 'USI', 'LSI']

# Read residential Data

In [5]:
Res_covar = {
    'UNI': ['Mean Air Temperature in Celsius', 'Mean Vapour Pressure Hpa', 'Days Of Wind Gust >= 24 Knots in Day'],
    'CNI' : None,
    'LNI' : ['Mean Air Temperature in Celsius', 'Mean Vapour Pressure Hpa'],
    'USI' : ['Mean Air Temperature in Celsius', 'Mean Vapour Pressure Hpa', 'Mean Wind Speed M/Sec'],
    'LSI' : ['Mean Air Temperature in Celsius','Mean Vapour Pressure Hpa']
}

In [6]:
pred_files = '../BestNlinear'

files = os.listdir(pred_files)

In [7]:
residential = []
columns = 'Est. Total consumption (GWh)'
zone_list = []

for file in files:
    if 'res' in file:
        if len(residential) == 0:
            residential.append(pd.read_csv(pred_files + '/' + file))
            zone_list.append(file.split(',')[1])
        else:
            zone_data = pd.read_csv(pred_files + '/' + file)
            residential.append(zone_data[columns])
            zone_list.append(file.split(',')[1])



In [8]:
zone_list
residential = pd.concat(residential, axis=1)
residential.columns = ['Date'] + zone_list
residential = TimeSeries.from_dataframe(residential, time_col="Date")

# Read in business

In [9]:
bus_covar = {
    'UNI': ['Export prices', 'Import values', 'Export values'],
    'CNI' : ['Export prices', 'Import values', 'Export values'],
    'LNI' : ['Export prices', 'Export values','Mean Air Temperature in Celsius','Mean Vapour Pressure Hpa'],
    'USI' : None,
    'LSI' : ['Mean Air Temperature in Celsius','Mean Vapour Pressure Hpa', 'Mean Wind Speed M/Sec']
}



In [10]:
business = []
columns = 'Selected business demand (GWh)'
zone_list_bus = []

for file in files:
    if 'bus' in file:
        if len(business) == 0:
            business.append(pd.read_csv(pred_files + '/' + file))
            zone_list_bus.append(file.split(',')[1])
        else:
            zone_data = pd.read_csv(pred_files + '/' + file)
            business.append(zone_data[columns])
            zone_list_bus.append(file.split(',')[1])


business = pd.concat(business, axis=1)
business.columns = ['Date'] + zone_list_bus
business = TimeSeries.from_dataframe(business, time_col="Date")



# Evaluation

### Residential

In [11]:
residential_metrics = pd.DataFrame(columns=['RMSE', 'MAE', 'MAPE', 'Zone'])
desc = 'Est. Total consumption (GWh)'

for i in zone_list:
    # Get the true data
    truth_index = TRUE_ORDER.index(i)
    true_data = timeseries_list[truth_index]

    # Split to pred data
    start_pred = pd.Timestamp("2018-01-01")
    train, val = true_data.split_after(start_pred)

    # Get the zone data
    ypred = residential[i]
    
    residential_metrics.loc[len(residential_metrics.index)] = [rmse(val[desc], ypred), 
                                                               mae(val[desc], ypred), 
                                                               mape(val[desc], ypred), 
                                                                i,]
    

In [12]:
residential_metrics

,RMSE,MAE,MAPE,Zone
0,10.649168,8.613445,3.727516,CNI
1,10.947812,8.307235,4.060857,LNI
2,6.837435,5.210282,4.378534,LSI
3,23.851384,17.727212,4.585845,UNI
4,12.760862,9.560666,4.550774,USI


In [13]:
Res_covar

{'UNI': ['Mean Air Temperature in Celsius',
  'Mean Vapour Pressure Hpa',
  'Days Of Wind Gust >= 24 Knots in Day'],
 'CNI': None,
 'LNI': ['Mean Air Temperature in Celsius', 'Mean Vapour Pressure Hpa'],
 'USI': ['Mean Air Temperature in Celsius',
  'Mean Vapour Pressure Hpa',
  'Mean Wind Speed M/Sec'],
 'LSI': ['Mean Air Temperature in Celsius', 'Mean Vapour Pressure Hpa']}

### Business

In [14]:
business_metrics = pd.DataFrame(columns=['RMSE', 'MAE', 'MAPE', 'Zone'])
desc = 'Selected business demand (GWh)'

for i in zone_list_bus:
    # Get the true data
    truth_index = TRUE_ORDER.index(i)
    true_data = timeseries_list[truth_index]

    # Split to pred data
    start_pred = pd.Timestamp("2018-01-01")
    train, val = true_data.split_after(start_pred)

    # Get the zone data
    ypred = business[i]
    
    business_metrics.loc[len(business_metrics.index)] = [rmse(val[desc], ypred), 
                                                               mae(val[desc], ypred), 
                                                               mape(val[desc], ypred), 
                                                                i,]
    

In [15]:
business_metrics

,RMSE,MAE,MAPE,Zone
0,27.774127,18.980568,4.395407,CNI
1,13.261595,10.232979,4.084249,LNI
2,21.578079,17.554347,2.722862,LSI
3,40.334553,28.429646,5.073832,UNI
4,28.644798,20.717688,7.036064,USI


In [16]:
bus_covar

{'UNI': ['Export prices', 'Import values', 'Export values'],
 'CNI': ['Export prices', 'Import values', 'Export values'],
 'LNI': ['Export prices',
  'Export values',
  'Mean Air Temperature in Celsius',
  'Mean Vapour Pressure Hpa'],
 'USI': None,
 'LSI': ['Mean Air Temperature in Celsius',
  'Mean Vapour Pressure Hpa',
  'Mean Wind Speed M/Sec']}